In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Electra 모델 클래스
class ElectraVectorizer:
    def __init__(self, model_name):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)

    def get_last_hidden_state(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state[0].cpu().numpy()  # 첫 번째 hidden state 벡터를 사용

# 데이터 로드
diary_text = "오늘 하루는 정말 즐거웠다."  # 예시 일기 텍스트
books_df = pd.read_excel("/content/book_list_with_emotions.xlsx")  # 도서 데이터 로드
book_texts = books_df['introduction2'].tolist()

# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer = ElectraVectorizer(model_name)

# 일기 텍스트와 도서 텍스트를 벡터로 변환
diary_vector = vectorizer.get_last_hidden_state(diary_text)
book_vectors = [vectorizer.get_last_hidden_state(text) for text in book_texts]

# 코사인 유사도 계산
similarities = cosine_similarity(diary_vector, book_vectors).flatten()

# 유사도가 높은 상위 3개의 도서 선택
top_indices = similarities.argsort()[-3:][::-1]
top_books = books_df.iloc[top_indices]

# 결과 출력
print("유사도가 높은 상위 3개의 도서:")
print(top_books[['title','author','category', 'introduction2']])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/815k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/452M [00:00<?, ?B/s]

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (972,) + inhomogeneous part.

In [17]:
diary_vector

array([[ 1.09445170e-01, -4.36338753e-01, -4.28789526e-01,
        -2.98077911e-01,  2.02738285e-01,  4.42556918e-01,
        -4.14739251e-01,  6.12284355e-02,  1.10166466e+00,
        -4.69370633e-01, -6.08086228e-01, -1.82652518e-01,
        -1.10119335e-01, -6.40796185e-01, -2.99949408e-01,
         5.02734333e-02,  2.60855675e-01, -2.58010119e-01,
        -4.96660441e-01,  2.09521726e-01,  4.61885095e-01,
         1.39749944e-01,  6.82271659e-01,  1.99511379e-01,
        -5.35970517e-02,  8.54797125e-01,  1.76432565e-01,
         3.50184470e-01, -4.94004786e-01, -1.44155532e-01,
         3.84781659e-01, -4.39458042e-01,  2.73058623e-01,
         1.50929779e-01, -2.39477709e-01, -2.77536720e-01,
         2.97962334e-02, -4.53093052e-01, -5.19030869e-01,
        -2.58541442e-02, -5.93824565e-01, -8.89550447e-01,
        -7.68762171e-01,  4.03862685e-01,  3.92335206e-01,
        -4.90856245e-02, -6.15957379e-01,  6.41828537e-01,
         5.26393466e-02, -3.96612436e-01, -4.74125966e-0

In [15]:
book_vectors[0]

array([[ 7.37343609e-01, -9.44546685e-02, -5.65731466e-01,
        -2.50355691e-01, -2.34553859e-01,  1.77614074e-02,
         1.01151340e-01,  2.56758500e-02,  1.15801714e-01,
        -1.21188248e-02, -1.62387550e-01, -1.09105920e-02,
         2.95278169e-02, -2.41972759e-01,  4.44271713e-02,
         9.72918719e-02,  1.01625621e-01, -3.16964060e-01,
         4.11236025e-02,  1.73202306e-01, -1.32539451e-01,
        -2.23534331e-01,  3.11072618e-01,  4.11741912e-01,
        -6.95257038e-02,  2.37485573e-01, -7.34323934e-02,
        -1.04369931e-02, -1.13389045e-01,  4.33971882e-02,
         1.40794918e-01, -1.08867712e-01,  2.21336827e-01,
         1.91424400e-01, -1.53277472e-01, -3.95537615e-01,
         1.58275161e-02, -1.91550776e-01, -1.52564734e-01,
        -1.82612643e-01,  2.61656523e-01, -3.07867914e-01,
        -1.82272956e-01, -4.75257337e-01, -7.52578303e-02,
         8.06688592e-02,  5.12114204e-02, -6.53513744e-02,
         2.69862432e-02, -2.37725049e-01, -3.66142929e-0

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Electra 모델 클래스
class ElectraVectorizer:
    def __init__(self, model_name):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)

    def get_first_hidden_state(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        #tokens = vectorizer.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        #print("Tokens:", tokens)
        #print("Token IDs:", inputs['input_ids'][0])
        with torch.no_grad():
            outputs = self.model(**inputs)
            #print(outputs.last_hidden_state[0].size())
        return outputs.last_hidden_state[0][0].cpu().numpy()  # 첫 번째 hidden state 벡터를 사용

# 데이터 로드
books_df = pd.read_excel("/content/book_list_with_emotions.xlsx")  # 도서 데이터 로드
book_texts = books_df['introduction2'].tolist()

# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer = ElectraVectorizer(model_name)

# 도서 벡터 미리 계산하여 저장
book_vectors = [vectorizer.get_first_hidden_state(text) for text in book_texts]


In [3]:
# 일기 벡터 계산
diary_text = "나 너무 화나. 골치가 아파."
diary_vector = vectorizer.get_first_hidden_state(diary_text)
# 코사인 유사도 계산 및 상위 3개의 도서 선택
similarities = []
idx = 0
for book_vector in book_vectors:
    similarity = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities.append((similarity[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 도서 선택
top_indices = sorted(similarities, key=lambda x : (x[0], x[1]))
top_indices = [top_indices[-i][1] for i in range(1, 6)]
top_books = books_df.iloc[top_indices]

for index in top_indices:
  print(similarities[index])

# 결과 출력
print("유사도가 높은 상위 3개의 도서:")
print(top_books[['title','author','category', 'introduction2','emotion']])

(0.7709701, 212)
(0.6990696, 412)
(0.6896875, 720)
(0.6812705, 211)
(0.677607, 566)
유사도가 높은 상위 3개의 도서:
          title            author category  \
212      완전한 행복             정유정 저  한국 장편소설   
412          구토  장 폴 사르트르 저/임호경 역  서양 고전문학   
720     생의 한가운데          루이제 린저 저     독일소설   
211  슬픔이 택배로 왔다             정호승 저     한국 시   
566        모비 딕     허먼 멜빌 저/김석희 역  서양 고전문학   

                                         introduction2 emotion  
212  완전한 행복은 나르시시스트가 자신의 행복을 위해 타인의 삶을 휘두르기 시작하면서 벌...      분노  
412   앙투안 로캉탱은 역사 연구를 위해 부빌이라는 도시에 정착하고 그곳에서 일기 형식으...      분노  
720  주인공 슈타인은 니나를 사랑하며 그의 일기와 편지에서 그 감정을 표현합니다 니나와 ...      분노  
211  정호승 시인의 이번 시집은 죽음에 대한 사유가 중심을 이룬다 시인은 죽음을 삶의 일...      분노  
566  에이해브 선장은 거대한 흰 고래 모비 딕을 잡기 위해 포경선 피쿼드호를 이끌고 바다...      중립  


In [4]:
top_books

,title,introduction,author,category,image_url,link,introduction2,emotion
212,완전한 행복,행복은 덧셈이 아니야 행복은 뺄셈이야 완전해질 때까지 불행의 가능성을 없애가는 거 ...,정유정 저,한국 장편소설,https://image.yes24.com/goods/101908580/XL,https://www.yes24.com/Product/Goods/101908580,완전한 행복은 나르시시스트가 자신의 행복을 위해 타인의 삶을 휘두르기 시작하면서 벌...,분노
412,구토,사르트르의 모든 사유는 구토에서 흘러나왔고 구토로 흘러든다 세기 프랑스의 대표적 지...,장 폴 사르트르 저/임호경 역,서양 고전문학,https://image.yes24.com/goods/96651059/XL,https://www.yes24.com/Product/Goods/96651059,앙투안 로캉탱은 역사 연구를 위해 부빌이라는 도시에 정착하고 그곳에서 일기 형식으...,분노
720,생의 한가운데,인생을 살아가면서 겪게 되는 사랑과 좌절과 생에 대한 집념이 응축되어 나타난 시대를...,루이제 린저 저,독일소설,https://image.yes24.com/momo/TopCate486/MidCat...,https://www.yes24.com/Product/Goods/109766,주인공 슈타인은 니나를 사랑하며 그의 일기와 편지에서 그 감정을 표현합니다 니나와 ...,분노
211,슬픔이 택배로 왔다,사랑하기에도 너무 짧은 인생 비워내는 마음에 관한 시편들 문학평론가 이성혁이 해설 ...,정호승 저,한국 시,https://image.yes24.com/goods/113420159/XL,https://www.yes24.com/Product/Goods/113420159,정호승 시인의 이번 시집은 죽음에 대한 사유가 중심을 이룬다 시인은 죽음을 삶의 일...,분노
566,모비 딕,최고의 번역으로 선보이는 무삭제 완역 일러스트판 탐색과 추구 투쟁과 파멸이 얽힌 전...,허먼 멜빌 저/김석희 역,서양 고전문학,https://image.yes24.com/goods/3677815/XL,https://www.yes24.com/Product/Goods/3677815,에이해브 선장은 거대한 흰 고래 모비 딕을 잡기 위해 포경선 피쿼드호를 이끌고 바다...,중립


In [7]:
top_books['introduction2']

212                           완전한 행복은 나르시시스트가 자신의 행복을 위해 타인의 삶을 휘두르기 시작하면서 벌어지는 일상의 악을 다룬다 소설은 세 명의 시점을 교차하며 행복을 위해 방해가 되는 것들을 제거해 나가는 주인공의 노력이 어떤 결말을 맞이하는지를 보여준다 이를 통해 작가는 무해하고 무결한 행복에 대한 사회적 강박증에 대해 묵직한 질문을 던진다
412     앙투안 로캉탱은 역사 연구를 위해 부빌이라는 도시에 정착하고 그곳에서 일기 형식으로 자신의 체험을 기록한다 로캉탱은 일상 속에서 반복적으로 구토라는 불쾌한 감정을 느끼며 이는 그의 존재와 실존에 대한 깊은 고뇌를 반영한다 결국 로캉탱은 문학을 통해 구토를 극복하고 진정한 삶으로 구원받으려는 모습을 그려내며 이는 사르트르의 실존주의 철학을 문학적으로 형상화한 것이다
720                                                                                                      주인공 슈타인은 니나를 사랑하며 그의 일기와 편지에서 그 감정을 표현합니다 니나와 그녀의 언니는 며칠 동안 짧은 만남과 대화를 나눕니다 이 이야기는 사랑과 좌절 생에 대한 집념을 담고 있습니다
211                                                                            정호승 시인의 이번 시집은 죽음에 대한 사유가 중심을 이룬다 시인은 죽음을 삶의 일부로 받아들이며 비움과 사랑을 통해 삶의 고통과 증오에서 벗어나려 한다 또한 시집 곳곳에는 시인의 개인적인 일화와 가족에 대한 감동적인 이야기들이 담겨 있다
566                                        에이해브 선장은 거대한 흰 고래 모비 딕을 잡기 위해 포경선 피쿼드호를 이끌고 바다로 나선다 이 항해를 통해 선원들은 다양한 고래와 포경업에 대한 지식을 습득하며 에이해브의 집착과 광기가 

In [6]:
# 출력 제한 설정 해제
pd.set_option('display.max_colwidth', None)


In [30]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


int2emo = {0: "기쁨", 1: "당황",  2: "분노", 3:"불안", 4:"상처", 5: "슬픔", 6: "중립"}
class ElectraSentimentClassifier:
    def __init__(self):
        self.model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name).to(self.device)

    def predict_sentiment(self, conversation):
        inputs = self.tokenizer(conversation, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).item()
        probabilities = torch.nn.functional.softmax(logits, dim=-1)[0].tolist()
        self.print_result(conversation, probabilities, predictions)

        return predictions, probabilities  # Ensure int2emo[predictions] is returned correctly

    def ready_model(self):
        self.model.to(self.device)

    def unload_model(self):
        self.model.cpu()
        torch.cuda.empty_cache()
        del self.model
        self.model = None

    def print_result(self, sentence, probabilities, predictions):
        # 터미널에 출력
        print("==== Model Prediction ====")
        print(f"Input Sentence: {sentence}")
        print("Probabilities:")
        for i, prob in enumerate(probabilities):
            print(f"  {int2emo[i]}: {prob * 100:.1f}%")
        predicted_label = int2emo[predictions]  # Add this line to define predicted_label
        print(f"Predicted Label: {predicted_label} ({predictions})")
        print("==========================")

In [31]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Electra 모델 클래스
class ElectraVectorizer:
    def __init__(self, model_name):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)

    def get_first_hidden_state(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        #tokens = vectorizer.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        #print("Tokens:", tokens)
        #print("Token IDs:", inputs['input_ids'][0])
        with torch.no_grad():
            outputs = self.model(**inputs)
            #print(outputs.last_hidden_state[0].size())
        return outputs.last_hidden_state[0][0].cpu().numpy()  # 첫 번째 hidden state 벡터를 사용


In [181]:
# 일기 벡터 계산
diary_text = "오늘 학교에서 그동안 몰랐던 친구에게 고백을 받았다. 너무 놀라서 잠시 말을 잇지 못했지만, 진심 어린 고백에 마음이 따뜻해졌다. 앞으로 어떻게 해야 할지 고민이 되면서도 설레는 하루였다."
diary_vector = vectorizer.get_first_hidden_state(diary_text)

In [118]:
# 감정 분석 클래스 인스턴스 생성
classifier = ElectraSentimentClassifier()

# 사용 예제
prediction, _ = classifier.predict_sentiment(diary_text)
emotion = int2emo[prediction]

print(f"감정: {emotion}")

==== Model Prediction ====
Input Sentence: 오늘 학교에서 그동안 몰랐던 친구에게 고백을 받았다. 너무 놀라서 잠시 말을 잇지 못했지만, 진심 어린 고백에 마음이 따뜻해졌다. 앞으로 어떻게 해야 할지 고민이 되면서도 설레는 하루였다.
Probabilities:
  기쁨: 98.6%
  당황: 0.3%
  분노: 0.0%
  불안: 0.1%
  상처: 0.1%
  슬픔: 0.5%
  중립: 0.4%
Predicted Label: 기쁨 (0)
감정: 기쁨


In [54]:
# 데이터 로드
books_df = pd.read_excel('/content/book_list_with_category_filtered.xlsx')  # 책 데이터 로드
book_texts = books_df['introduction2'].tolist()

# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer = ElectraVectorizer(model_name)

# 도서 벡터 미리 계산하여 저장
book_vectors = [vectorizer.get_first_hidden_state(text) for text in book_texts]

In [126]:
# 코사인 유사도 계산 및 상위 3개의 도서 선택
similarities = []
idx = 0
for book_vector in book_vectors:
    similarity = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities.append((similarity[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 도서 선택
top_indices = sorted(similarities, key=lambda x : (x[0], x[1]))
top_indices = [top_indices[-i][1] for i in range(1, 6)]
top_books = books_df.iloc[top_indices]

for index in top_indices:
  print(similarities[index])

# 결과 출력
print("유사도가 높은 상위 3개의 도서:")
print(top_books[['title','author','category', 'introduction2','emotion']])

(0.9500073, 376)
(0.94282067, 503)
(0.93708396, 244)
(0.9353216, 394)
(0.93519926, 232)
유사도가 높은 상위 3개의 도서:
                title                author category  \
376         서부전선 이상없다  에리히 마리아 레마르크 저/홍성광 역     독일소설   
503            작별 곁에서                 신경숙 저  한국 단편소설   
244                속죄        이언 매큐언 저/한정아 역  영미 장편소설   
394  백만장자를 위한 공짜 음식 2           이민진 저/유소영 역  영미 장편소설   
232          세일즈맨의 죽음               아서 밀러 저  영미 장편소설   

                                                                                                                                                                                                                  introduction2  \
376                                                                                               주인공 파울 보이머는 담임선생의 권유로 반 친구들과 함께 전쟁에 입대한다 전장에서는 생존과 기본적인 욕구 외에는 아무것도 중요하지 않은 기계로 변해간다 결국 친구들 모두가 죽음을 피할 수 없으며 전우애조차도 허망하게 느껴진다   
503  뉴욕에 체류 중인 화자는 연락이 끊긴 고국의 화가에게 편지를 쓰며 고국으로 돌아가지 못한 채 살아온 자신의 삶을 되돌아본다 암투병 중인 오랜 친구와의 작별을 위해 독일로 향하

In [127]:
# 반대 코사인 유사도 계산 및 상위 3개의 도서 선택
similarities = []
idx = 0
for book_vector in book_vectors:
    similarity = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities.append((similarity[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 도서 선택
top_indices = sorted(similarities, key=lambda x : (x[0], x[1]))
top_indices = [top_indices[i][1] for i in range(1, 6)]
top_books = books_df.iloc[top_indices]

for index in top_indices:
  print(similarities[index])

# 결과 출력
print("유사도가 높은 상위 3개의 도서:")
print(top_books[['title','author','category', 'introduction2','emotion']])

(0.027028311, 427)
(0.027929703, 401)
(0.029294565, 239)
(0.065106854, 163)
(0.06726534, 253)
유사도가 높은 상위 3개의 도서:
                      title                          author category  \
427             서머싯 몸 단편선 1               윌리엄 서머싯 몸 저/황소연 역  영미 단편소설   
401                  갈매기의 꿈                        리처드 바크 저  영미 장편소설   
239                 클라라와 태양                가즈오 이시구로 저/홍한별 역  영미 장편소설   
163     가장 아름다운 기억을 너에게 보낼게                하세가와 카오리 저/김진환 역  일본 장편소설   
253  거울 나라의 앨리스 초판본 리커버 디자인  루이스 캐롤 저/존 테니얼 그림/양윤정 해설/성세희 역  영미 장편소설   

                                                                                                                                                                                                          introduction2  \
427  서머싯 몸은 간결하면서도 재미있게 이야기를 풀어내며 유머와 서스펜스를 통해 인간의 양면성과 모호한 현실을 드러낸다 그의 단편 소설은 예상을 뛰어넘는 행동과 반전으로 극적 긴장감을 유지하며 다양한 시공간과 인물들을 섬세하게 묘사해 독자에게 생생한 독서 경험을 선사한다 몸의 소설은 인간 본성의 모순에서 즐거움을 찾아내며 연민과 이해를 바탕으로 따뜻한 웃음을 끌어내는 작품이 많다   
401             

In [130]:
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format
import numpy as np
import random  # 랜덤 모듈 임포트
import os

import logging

# 로그 생성
logger = logging.getLogger()

# 로그의 출력 기준 설정
logger.setLevel(logging.INFO)

# log 출력 형식
formatter = logging.Formatter("%(message)s")

# Start defining and assigning your handlers here
handler = logging.StreamHandler()
handler.setFormatter(formatter)
logger.addHandler(handler)

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# 도서 목록 로드
file_path = '/content/movie_dataset_image.csv'  # 파일 업로드 후 경로 변경 필요
book_list = pd.read_csv(file_path, sep=',')

# 감정 분석 클래스 인스턴스 생성
classifier = ElectraSentimentClassifier()

# 도서 소개 텍스트 감정 분석
def analyze_book_emotions(book_list):
    book_emotions = {}
    emotions = []
    for index, row in book_list.iterrows():
        introduction = row['text']
        prediction, _ = classifier.predict_sentiment(introduction)
        book_emotions[row['text']] = int2emo[prediction]
        emotions.append(int2emo[prediction])
    book_list['emotion'] = emotions
    return book_emotions

# 도서 소개 텍스트 감정 분석 결과
book_emotions = analyze_book_emotions(book_list)


print(f"감정: {emotion}")
print("추천 도서:")

# 결과를 엑셀 파일로 저장
output_file_path = '/content/movie_with_emotions.xlsx'
book_list.to_excel(output_file_path, index=False)

==== Model Prediction ====
Input Sentence: 가리봉동 소탕작전 후 4년 뒤, 금천서 강력반은 베트남으로 도주한 용의자를 인도받아 오라는 미션을 받는다. 괴물형사 ‘마석도’(마동석)와 ‘전일만’(최귀화) 반장은 현지 용의자에게서 수상함을 느끼고, 그의 뒤에 무자비한 악행을 벌이는 ‘강해상’(손석구)이 있음을 알게 된다. ‘마석도’와 금천서 강력반은 한국과 베트남을 오가며 역대급 범죄를 저지르는 ‘강해상’을 본격적으로 쫓기 시작하는데... 나쁜 놈들 잡는 데 국경 없다! 통쾌하고 화끈한 범죄 소탕 작전이 다시 펼쳐진다!
Probabilities:
  기쁨: 91.0%
  당황: 0.5%
  분노: 1.8%
  불안: 0.1%
  상처: 0.1%
  슬픔: 1.3%
  중립: 5.2%
Predicted Label: 기쁨 (0)
==== Model Prediction ====
Input Sentence: 대국민 오디션 이후 각자의 자리에서 꿈을 이루고 있는 버스터 문(매튜 맥커너히)과 크루들에게 레드 쇼어 시티에서 전 세계가 주목하는 사상 최고의 쇼가 펼쳐진다는 소식이 들려오고 버스터 문과 크루들은 도전에 나선다.  그러나 최고의 스테이지에 서기 위한 경쟁은 이전과는 비교도 할 수 없을 만큼 치열하고, 버스터 문은 완벽한 라이브를 위해 종적을 감춘 레전드 뮤지션 클레이(보노)를 캐스팅하겠다는 파격 선언을 하는데!
Probabilities:
  기쁨: 17.4%
  당황: 4.2%
  분노: 31.6%
  불안: 19.7%
  상처: 3.4%
  슬픔: 22.9%
  중립: 0.8%
Predicted Label: 분노 (2)
==== Model Prediction ====
Input Sentence: 혈귀로 변한 여동생 ‘네즈코’를 되돌리기 위해 귀살대가 된 ‘탄지로’!  어둠 속을 달리는 무한열차에서 승객들이 흔적 없이 사라진다는 소식에 ‘젠이츠’, ‘이노스케’와 함께 임무 수행을 위해 무한열차에 탑승한다. 그리고 그 곳에서 만난 귀살

In [131]:
# 데이터 로드
books_df2 = pd.read_excel('/content/movie_with_emotions.xlsx')  # 책 데이터 로드
book_texts2 = books_df2['text'].tolist()

# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer2 = ElectraVectorizer(model_name)

# 도서 벡터 미리 계산하여 저장
book_vectors2 = [vectorizer2.get_first_hidden_state(text) for text in book_texts2]

In [183]:
# 코사인 유사도 계산 및 상위 3개의 도서 선택
similarities2 = []
idx = 0
for book_vector in book_vectors2:
    similarity2 = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities2.append((similarity2[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 도서 선택
top_indices2 = sorted(similarities2, key=lambda x : (x[0], x[1]))
top_indices2 = [top_indices2[-i][1] for i in range(1, 6)]
top_books2 = books_df2.iloc[top_indices2]

for index in top_indices2:
  print(similarities2[index])

# 결과 출력
print("유사도가 높은 상위 3개의 도서:")
print(top_books2[['title','text','emotion']])

(0.82725644, 73)
(0.8064281, 27)
(0.79773855, 28)
(0.79479957, 237)
(0.79151565, 160)
유사도가 높은 상위 3개의 도서:
                     title  \
73                  사울의 아들   
27   루이스 웨인: 사랑을 그린 고양이 화가   
28                  날씨의 아이   
237                  안녕하세요   
160                    챔피언   

                                                                                                                                                                                                                                                                                                text  \
73          시체 처리반으로 일하던 남자 앞에 오늘, 아들의 주검이 도착했다… 나치의 만행이 극에 달했던 1944년, 아우슈비츠 수용소에는 시체들을 처리하기 위한 비밀 작업반이 있었다. ‘존더코만도’라 불리던 이들은 X자 표시가 된 작업복을 입고 아무 것도 묻지 않고 오직 시키는 대로 주어진 임무를 수행한다. 그러던 어느 날, ‘존더코만도’ 소속이었던 남자 ‘사울’의 앞에 어린 아들의 주검이 도착한다. 처리해야 할 시체더미들 사이에서 아들을 빼낸 ‘사울’은 랍비를 찾아 제대로 된 장례를 치러주기로 결심하는데…   
27                                                                                                                    모든 동

In [182]:
# 코사인 유사도 계산 및 하위 3개의 도서 선택
similarities2 = []
idx = 0
for book_vector in book_vectors2:
    similarity2 = cosine_similarity(diary_vector.reshape(1, -1), book_vector.reshape(1, -1))
    similarities2.append((similarity2[0][0], idx))
    idx += 1

# 유사도가 높은 상위 3개의 도서 선택
top_indices2 = sorted(similarities2, key=lambda x : (x[0], x[1]))
top_indices2 = [top_indices2[i][1] for i in range(1, 6)]
top_books2 = books_df2.iloc[top_indices2]

for index in top_indices2:
  print(similarities2[index])

# 결과 출력
print("유사도가 높은 상위 3개의 도서:")
print(top_books2[['title','text','emotion']])

(-0.13900939, 260)
(-0.09611, 271)
(-0.08634038, 46)
(-0.07654515, 65)
(-0.07350642, 62)
유사도가 높은 상위 3개의 도서:
         title  \
260        셧 인   
271      더 스퀘어   
46   트리 오브 라이프   
65         25시   
62          히든   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                           text  \
260                                                                                                                                                                                                                                                     죽은 할머니의 오래된 집을 정리하고 떠나려던 '제시카' 앞에 불청객인 마약 중독자 전 남편과 그의 친구 ‘새미’가 들이닥친